Workflow:
* find random point in the skeleton
* get random orientation O_t of the point
* calculate the MVF around the point, using distance, label and apply gaussian filter to the MVF
* apply the MVF to image
* using weighted average
* using FBP to get simulated image

In [1]:
# load image
from monai.transforms import LoadImage
from skimage.morphology import skeletonize
import numpy as np
from scipy.ndimage import sobel
from post_processing.fracture_detection import get_point_orientation
from slicer_visulization.slicer_mvf import save_mvf, save_image
from slicer_visulization.slicer_mark_up import create_extended_plane_markup_json
from scipy.ndimage import gaussian_filter
from scipy.ndimage import map_coordinates
from monai.transforms import SaveImage
from batchgenerators.augmentations.utils import create_zero_centered_coordinate_mesh
import pathlib


def find_random_one_numpy(image):
    # 找到值为1的所有点的坐标
    ones_indices = np.where(image == 1)
    # 转换坐标为列表形式 [(x1, y1), (x2, y2), ...]
    ones_list = list(zip(ones_indices[0], ones_indices[1], ones_indices[2]))

    if not ones_list:
        return None  # 如果没有找到值为1的点，则返回None

    # 从列表中随机选择一个点
    return ones_list[np.random.randint(len(ones_list))]


def calculate_normal_vector(image, point):
    # 从点的坐标中获取x, y, z
    x, y, z = point
    # 获取图像的大小
    x_max, y_max, z_max = image.shape
    # 计算梯度
    dx = (image[min(x + 1, x_max - 1), y, z] - image[max(x - 1, 0), y, z]) / 2
    dy = (image[x, min(y + 1, y_max - 1), z] - image[x, max(y - 1, 0), z]) / 2
    dz = (image[x, y, min(z + 1, z_max - 1)] - image[x, y, max(z - 1, 0)]) / 2
    # 归一化
    norm = np.sqrt(dx ** 2 + dy ** 2 + dz ** 2)
    return dx / norm, dy / norm, dz / norm


def generate_random_vector_perpendicular_to(n):
    # 确保n是单位向量
    n = n / np.linalg.norm(n)

    # 生成一个随机向量
    r = np.random.rand(3)

    # 计算垂直于n的向量
    v = np.cross(n, r)

    # 如果v是零向量（非常罕见的情况，但理论上可能如果r和n平行），重新生成r
    while np.linalg.norm(v) == 0:
        r = np.random.rand(3)
        v = np.cross(n, r)

    # 标准化v
    v = v / np.linalg.norm(v)

    return v


def calculate_markup_orientation_from_normal(normal, up_vector=None):
    """
    We need normal vector and up_vector are unit vectors
    """
    # 将法向量转换为单位向量
    normal_vector = np.array(normal)
    normal_vector /= np.linalg.norm(normal_vector)
    if up_vector is not None:
        # 选择一个“向上”向量，这里我们选择Y轴
        up_vector = np.array(up_vector)
        up_vector /= np.linalg.norm(up_vector)
    else:
        up_vector = np.array([0, 0, 1])

    # 计算右向量，使其垂直于法向量和“向上”向量
    # 防止“向上”向量和法向量共线
    if np.abs(np.dot(up_vector, normal_vector)) > 0.99:
        orientation_matrix = np.eye(3)
        return orientation_matrix
    right_vector = np.cross(up_vector, normal_vector)
    right_vector /= np.linalg.norm(right_vector)

    # 重新计算调整后的“向上”向量，确保它垂直于法向量和右向量
    up_vector_adjusted = np.cross(normal_vector, right_vector)

    # 构建orientation矩阵
    orientation_matrix = np.vstack([right_vector, up_vector_adjusted, normal_vector]).T

    return orientation_matrix


def image2world(point, spacing, origin, direction):
    """
    Convert point from image coordinate to world coordinate
    """
    # 输入的是list
    direction_com = np.array([direction[0], direction[4], direction[8]])
    return (np.array(point) * np.array(spacing) * np.array(direction_com) + np.array(origin)).tolist()


def vessel_motion_simulator(image, label, heart, seed, scale_list):
    # 1. Get the skeleton of the label
    skeleton = skeletonize(label) > 0

    # 2. Get the centerline of the skeleton
    centerline = np.zeros_like(skeleton)
    centerline[1:-1, 1:-1, 1:-1] = skeleton[1:-1, 1:-1, 1:-1] & ~skeleton[0:-2, 0:-2, 0:-2] & ~skeleton[2:, 2:, 2:]

    # 3. Get the normal vector of the centerline
    normal_vector = np.zeros_like(centerline, dtype=np.float32)
    for i in range(3):
        normal_vector[:, :, :, i] = sobel(centerline, axis=i)

    # 4. Get the orientation matrix of the centerline
    orientation_matrix = np.zeros((centerline.shape[0], centerline.shape[1], centerline.shape[2], 3, 3),
                                  dtype=np.float32)


H:\anaconda\Anaconda\envs\Pytorch_gpu\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


* calculate the MVF around the point, using distance, label and apply gaussian filter to the MVF

# Loading Data

In [4]:
loader = LoadImage(image_only=True)
image = loader(
    "H:/Graduate_project/segment_server/data/multi_phase_select/final_version_crop_train/test/main/img_crop/AI01_0043.nii.gz")
label = loader(
    "H:/Graduate_project/segment_server/data/multi_phase_select/final_version_crop_train/test/main/label_crop_clip/AI01_0043.nii.gz")
heart = loader(
    "H:/Graduate_project/segment_server/data/multi_phase_select/final_version_crop_train/test/main/heart_connect_crop/AI01_0043.nii.gz")
skeleton = skeletonize(label) > 0

# Coordinate system
original_affine = image.meta.get('original_affine')
direction_image = np.sign(original_affine[:3, :3].reshape(-1, )).tolist()
spacing = image.meta.get('pixdim')[1:4]
origin = original_affine[:3, 3]
sigma = 5  # gaussian kernel sigma for smoothing the orientation field
# scale_list = np.array([-1, -0.75, -0.5, -0.25, 0, 0.25, 0.5, 0.75, 1]) * sigma * 2  # scale factor for the random motion
scale_list = np.array([1]) * sigma * 2
seed_list = [6]
save_dir = "H:\slicer\motion_simulator"
pathlib.Path(save_dir).mkdir(parents=True, exist_ok=True)
for seed in seed_list:
    np.random.seed(seed)
    center = np.array(find_random_one_numpy(skeleton))[None,]
    orientation_centerline_spacing = \
        get_point_orientation(point=center, centerline=skeleton, spacing=spacing, size=3, is_end_point=False)[0]
    direction_image_com = np.array([direction_image[0], direction_image[4], direction_image[8]])
    orientation_centerline_world = orientation_centerline_spacing * direction_image_com
    orientation_centerline_image = orientation_centerline_spacing / spacing
    # randomize the orientation
    random_direction_3d = generate_random_vector_perpendicular_to(orientation_centerline_world)

    direction_plane = np.sign(original_affine[:3, :3].reshape(-1, )).tolist()
    center_world = image2world(point=center[0], spacing=spacing, origin=origin, direction=direction_plane)
    plane_normal = orientation_centerline_world.tolist()
    bounds = [-10.0, 10.0, -5.0, 5.0]
    coordinate_system = image.meta.get('space')

    markup_orientation = calculate_markup_orientation_from_normal(plane_normal).reshape(-1).tolist()

    markup_json_str = create_extended_plane_markup_json(center=center_world, normal=plane_normal, bounds=bounds,
                                                        coordinate_system=coordinate_system,
                                                        markup_orientation=markup_orientation,
                                                        orientation=direction_plane)
    # save to file
    with open(f"{save_dir}\plane_write_world_{seed}.mrk.json", "w") as f:
        f.write(markup_json_str)

    h, w, d = label.shape
    MVF_world = np.zeros((h, w, d, 3))
    MVF_world += random_direction_3d
    MVF_mask = np.zeros_like(label)
    center_point = center[0]
    region_size = 10
    x_l, x_u = max(center_point[0] - region_size, 0), min(center_point[0] + region_size, h)
    y_l, y_u = max(center_point[1] - region_size, 0), min(center_point[1] + region_size, w)
    z_l, z_u = max(center_point[2] - region_size, 0), min(center_point[2] + region_size, d)
    center_cube = slice(x_l, x_u), slice(y_l, y_u), slice(z_l, z_u)
    MVF_mask[center_cube] = label[center_cube]
    MVF_weight = gaussian_filter(MVF_mask, sigma=sigma)
    MVF_world = MVF_world * MVF_weight[..., None]
    MVF_image = (MVF_world / np.array(direction_image_com) / np.array(spacing))
    save_path = f"{save_dir}/mvf_plane_{seed}.nrrd"
    # 3D slicer visualization has a bug that it does not respect the affine matrix
    MVF_world[:, :, :, 0] = -MVF_world[:, :, :, 0]
    MVF_world[:, :, :, 1] = -MVF_world[:, :, :, 1]
    save_mvf(MVF_world, save_path=save_path, affine=image.meta["original_affine"],
             space=coordinate_system, scale_factor=10)
    print(f"plane_normal: {plane_normal}", f"random_direction_3d: {random_direction_3d}",
          f"dot product: {plane_normal @ random_direction_3d}")
    mean_image = np.zeros_like(image)
    mean_image = mean_image[None, ]
    warp_image_record = []
    for i, scale_factor in enumerate(scale_list):
        MVF_image_transpose = MVF_image.transpose(3, 0, 1, 2).astype(np.float32) * scale_factor
        tmp = tuple([np.arange(i) for i in (h, w, d)])
        coords = np.array(np.meshgrid(*tmp, indexing='ij')).astype(float)

        image_warp = map_coordinates(np.array(image), MVF_image_transpose + coords, order=1, mode='constant',
                                     cval=0)
        label_warp = map_coordinates(np.array(label), MVF_image_transpose + coords, order=0, mode='constant',
                                     cval=0)
        image_warp = image_warp[None, ]
        label_warp = label_warp[None, ]
        def name_func(x): return f"m{abs(x)}" if x < 0 else f"p{abs(x)}"
        saver = SaveImage(output_dir=save_dir, output_postfix=f"warp_image_seed_{seed}_scale_{name_func(scale_factor)}")
        saver(image_warp, meta_data=image.meta)
        saver = SaveImage(output_dir=save_dir, output_postfix=f"warp_label_seed_{seed}_scale_{name_func(scale_factor)}")
        saver(label_warp, meta_data=label.meta)
        warp_image_record.append(image_warp)
        print(scale_list)


    # weighted_list = np.ones_like(scale_list)
    # weighted_list_smooth = gaussian_filter(weighted_list, sigma=1, mode="constant")
    # for i in range(len(mean_image)):
    #     mean_image += weighted_list_smooth[i] * warp_image_record[i]
    # mean_image /= weighted_list_smooth.sum()
    # saver = SaveImage(output_dir=save_dir, output_postfix=f"warp_image_seed_{seed}_scale_mean")
    # saver(mean_image, meta_data=image.meta)

plane_normal: [-0.36927175002298585, 0.5768966098078007, -0.7285799038041246] random_direction_3d: [ 0.87304806 -0.05331706 -0.48471061] dot product: 6.938893903907228e-18
2024-03-06 21:21:53,360 INFO image_writer.py:194 - writing: H:\slicer\motion_simulator\AI01_0043\AI01_0043_warp_image_seed_6_scale_p10.nii.gz
2024-03-06 21:21:56,578 INFO image_writer.py:194 - writing: H:\slicer\motion_simulator\AI01_0043\AI01_0043_warp_label_seed_6_scale_p10.nii.gz
[10]


# Generate the MVF
* 三维图像中的方向, 图像的方向, 世界坐标系的方向， 世界坐标系方向不仅要考虑spacing，还要考虑图像的方向(-1, 0, 0, 0, -1, 0, 0, 0, 1)

### seed 5 have problem

# Warp the image and label

In [30]:
mean_image = np.zeros_like(image)[None, ]
weighted_list = np.ones_like(scale_list)
weighted_list_smooth = gaussian_filter(weighted_list, sigma=1, mode="constant")
weighted_list_smooth = np.array([0, 0, 0, 0, 0.5, 0, 0, 0, 0.5])
# weighted_list_smooth = weighted_list
weighted_list_smooth /= weighted_list_smooth.sum()
for i in range(len(weighted_list_smooth)):
    mean_image += warp_image_record[i] * weighted_list_smooth[i]
saver = SaveImage(output_dir=save_dir, output_postfix=f"warp_image_seed_{seed}_scale_mean_sigma_1")
saver(mean_image, meta_data=image.meta)

2024-02-28 11:22:20,606 INFO image_writer.py:194 - writing: H:\slicer\motion_artifact_simulator_scale_2\AI01_0043\AI01_0043_warp_image_seed_0_scale_mean_sigma_1.nii.gz


array([[[[   93.,    95.,   106., ...,   -73.,  -156.,  -244.],
         [   60.,    60.,    59., ...,    22.,   -23.,   -77.],
         [   15.,    12.,     8., ...,    67.,    56.,    36.],
         ...,
         [   37.,    30.,    24., ...,    48.,   -15.,  -102.],
         [   38.,    29.,    25., ...,    94.,    63.,   -12.],
         [   41.,    28.,    28., ...,   139.,   130.,    66.]],

        [[   93.,    90.,    96., ...,   -74.,  -158.,  -242.],
         [   64.,    62.,    57., ...,    19.,   -26.,   -76.],
         [   15.,    13.,     4., ...,    60.,    52.,    36.],
         ...,
         [   38.,    33.,    30., ...,   -65.,  -146.,  -243.],
         [   37.,    30.,    27., ...,     5.,   -49.,  -140.],
         [   36.,    25.,    22., ...,    63.,    34.,   -60.]],

        [[   89.,    91.,    93., ...,   -79.,  -163.,  -243.],
         [   66.,    64.,    57., ...,    17.,   -29.,   -77.],
         [   25.,    13.,     3., ...,    57.,    51.,    35.],
        